In [ ]:
import pandas as pd
import re
from genz_generation import Genz_generation
import csv


# Read the datasets
twitter_df = pd.read_csv('datasets/twitter_dataset.csv')

# Convert tweets to lowercase and combine all tweets into one string
twitter_df['Text'] = twitter_df['Text'].str.lower()

print(twitter_df['Text'])

## Getting API Keys 
```python
api_keys = [
    "your_groq_api_key_1",
    "your_groq_api_key_2",
    # Add more API keys as needed
]
```

### ***For sake of reference here's the link to get Groq API keys: [Groq API Keys](https://console.groq.com/keys)***

Replace the placeholder strings with your actual Groq API keys.

In [ ]:
# api_keys = [] your groq api keys list here


models = [
    "llama-3.3-70b-versatile",
    "llama3-70b-8192",
    "llama3-8b-8192",
    "meta-llama/llama-4-scout-17b-16e-instruct",
    
    "qwen-qwq-32b",
    "meta-llama/llama-4-maverick-17b-128e-instruct",
    "gemma2-9b-it",
    "llama-3.1-8b-instant",
    "mistral-saba-24b",

]

batch_size = 15


In [ ]:

model_index = 0
api_index = 0 
flag = 0
for i in range(0, len(twitter_df), batch_size):
    if flag == 1:
        break
    batch = twitter_df.iloc[i:i + batch_size]
    api_key = api_keys[api_index]
    api_index = (api_index + 1) % len(api_keys)  # Cycle through API keys
    print(f"Using API key: {api_key} for batch starting at index {i}")
    current_batch_df = pd.DataFrame(columns=['original', 'generated'])  # Temp for current batch
    
    for j in range(len(batch)):
        genz_content = Genz_generation(batch.iloc[j]['Text'], groq_api_key=api_key,model = models[model_index])
        if "Failed to generate" in genz_content:
            model_index = (model_index + 1) % len(models)
            print(f"Switching to model: {models[model_index]} for batch starting at index {i}")
            genz_content = Genz_generation(batch.iloc[j]['Text'], groq_api_key=api_key, model=models[model_index])
            if "Failed to generate" in genz_content:
                flag = 1
                break
        new_row = {'original': batch.iloc[j]['Text'], 'generated': genz_content}
        current_batch_df = pd.concat([current_batch_df, pd.DataFrame([new_row])], ignore_index=True)
    
    # Determine mode and header
    write_header = i == 0  # Only write header on first batch
    current_batch_df.to_csv('datasets/genz_data.csv', mode='a', index=False, header=write_header)

    print(f"Processed batch {i // batch_size + 1} of {len(twitter_df) // batch_size}.")
